In [ ]:
# 📓 traditional_backend_sid/face_detection_live.ipynb

import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import joblib

# === Step 1: Prepare Dataset from ./dataset/person_name/*.jpg ===
face_size = (100, 100)
faces = []
labels = []
dataset_dir = "../dataset"

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

for person_name in os.listdir(dataset_dir):
    person_folder = os.path.join(dataset_dir, person_name)
    if not os.path.isdir(person_folder):
        continue

    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)

        image = cv2.imread(img_path)
        if image is None:
            continue

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)

        faces_rect = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces_rect:
            face_roi = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, face_size)
            faces.append(face_resized.flatten())
            labels.append(person_name)

# === Step 2: Encode Labels ===
le = LabelEncoder()
y_encoded = le.fit_transform(labels)

# === Step 3: PCA (Eigenfaces) + Classifier (KNN) ===
pca = PCA(n_components=100, whiten=True, random_state=42)
x_pca = pca.fit_transform(faces)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_pca, y_encoded)

# Save the model for later (optional)
joblib.dump(pca, "pca_model.pkl")
joblib.dump(knn, "knn_model.pkl")
joblib.dump(le, "label_encoder.pkl")

print("[INFO] Model trained and saved. Ready for live demo.")

# === Live Detection & Recognition Loop ===
cap = cv2.VideoCapture(0)

print("[INFO] Press 'q' to exit the live feed...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    faces_live = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces_live:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, face_size).flatten().reshape(1, -1)
        face_pca = pca.transform(face_resized)
        prediction = knn.predict(face_pca)
        name = le.inverse_transform(prediction)[0]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow("Live Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("[INFO] Exiting live stream...")
        break

cap.release()
cv2.destroyAllWindows()



[INFO] Model trained and saved. Ready for live demo.


KeyboardInterrupt: 

: 